In [171]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sbs
from sklearn.preprocessing import OneHotEncoder, LabelBinarizer

In [155]:
data = pd.read_csv('churn.csv')
data = data.drop('customerID', axis=1)
len(data)

7043

In [156]:
t =(data['TotalCharges']==' ')
zer = [i for i, x in enumerate(t) if x]
data = data.drop(data.index[zer])
data['TotalCharges'] = pd.to_numeric(data['TotalCharges'])
len(data)

7032

In [158]:
data_sub = data.select_dtypes(include='O')

In [159]:
for i in data_sub:
    print(set(data[i]))

{'Male', 'Female'}
{'Yes', 'No'}
{'No', 'Yes'}
{'No', 'Yes'}
{'No', 'No phone service', 'Yes'}
{'No', 'Fiber optic', 'DSL'}
{'No', 'No internet service', 'Yes'}
{'Yes', 'No', 'No internet service'}
{'No', 'No internet service', 'Yes'}
{'No', 'No internet service', 'Yes'}
{'No', 'No internet service', 'Yes'}
{'No', 'No internet service', 'Yes'}
{'Two year', 'Month-to-month', 'One year'}
{'Yes', 'No'}
{'Credit card (automatic)', 'Electronic check', 'Mailed check', 'Bank transfer (automatic)'}
{'No', 'Yes'}


In [160]:
data.dtypes

gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges        float64
Churn                object
dtype: object

In [175]:
def dummyEncode(data):
        col = list(data.select_dtypes(include='object'))
        enc = LabelBinarizer()
        for i in col:
            try:
                data[i] = enc.transform(data[i])
            except:
                print('Error encoding '+i)
        return data

In [176]:
dummyEncode(data)

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,0,0,1,0,1,0,1,0,0,2,0,0,0,0,0,1,2,29.85,29.85,0
1,1,0,0,0,34,1,0,0,2,0,2,0,0,0,1,0,3,56.95,1889.50,0
2,1,0,0,0,2,1,0,0,2,2,0,0,0,0,0,1,3,53.85,108.15,1
3,1,0,0,0,45,0,1,0,2,0,2,2,0,0,1,0,0,42.30,1840.75,0
4,0,0,0,0,2,1,0,1,0,0,0,0,0,0,0,1,2,70.70,151.65,1
5,0,0,0,0,8,1,2,1,0,0,2,0,2,2,0,1,2,99.65,820.50,1
6,1,0,0,1,22,1,2,1,0,2,0,0,2,0,0,1,1,89.10,1949.40,0
7,0,0,0,0,10,0,1,0,2,0,0,0,0,0,0,0,3,29.75,301.90,0
8,0,0,1,0,28,1,2,1,0,0,2,2,2,2,0,1,2,104.80,3046.05,1
9,1,0,0,1,62,1,0,0,2,2,0,0,0,0,1,0,0,56.15,3487.95,0
